# EEG Data Preprocessing

## Converting Files to CSVs

Person1_Recording1_Person1Recording1.csv: annotation forward-filled.


C:\Users\aksel\AppData\Local\Temp\ipykernel_9540\3644778306.py:12: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Person1_Recording2_Person1Recording2.csv: annotation forward-filled.


C:\Users\aksel\AppData\Local\Temp\ipykernel_9540\3644778306.py:12: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Person1_Recording3_Person1Recording3.csv: annotation forward-filled.


C:\Users\aksel\AppData\Local\Temp\ipykernel_9540\3644778306.py:12: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Person2_Recording1_Person2Recording1.csv: annotation forward-filled.


=== INFO OM DET NYE DATASETTET (kombinert) ===

Shape: (761380, 39)
Kolonner: ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'Aux1', 'x_dir', 'y_dir', 'z_dir', 'time', 'annotation', 'source_csv']

-- HEAD(5) --
        Fp1        Fz        F3        F7       FT9       FC5       FC1  \
0 -0.341333 -0.341333 -0.028123 -0.341333 -0.000276 -0.006177 -0.341333   
1 -0.341333 -0.341333 -0.024354 -0.341333 -0.004998 -0.008506 -0.341333   
2 -0.341333 -0.341333 -0.020647 -0.341333 -0.010400 -0.012470 -0.341333   
3 -0.341333 -0.341333 -0.016728 -0.341333 -0.015081 -0.015459 -0.341333   
4 -0.341333 -0.341333 -0.014118 -0.341333 -0.017770 -0.016872 -0.341333   

         C3        T7       TP9  ...        F4        F8       Fp2      Aux1  \
0 -0.007168 -0.341333 -0.001292  ... -0.341333 -0.043082 -0.341333 -1.915041   
1 -0.008500 -0.3

Opprettet en stor CSV: 'csv_output/AllSelfmade_Combined.csv'
Dimensjoner: (761380, 39)
Eksempel på topp 5 rader:
         Fp1        Fz        F3        F7       FT9       FC5       FC1  \
0 -0.341333 -0.341333 -0.028123 -0.341333 -0.000276 -0.006177 -0.341333   
1 -0.341333 -0.341333 -0.024354 -0.341333 -0.004998 -0.008506 -0.341333   
2 -0.341333 -0.341333 -0.020647 -0.341333 -0.010400 -0.012470 -0.341333   
3 -0.341333 -0.341333 -0.016728 -0.341333 -0.015081 -0.015459 -0.341333   
4 -0.341333 -0.341333 -0.014118 -0.341333 -0.017770 -0.016872 -0.341333   

         C3        T7       TP9  ...        F4        F8       Fp2      Aux1  \
0 -0.007168 -0.341333 -0.001292  ... -0.341333 -0.043082 -0.341333 -1.915041   
1 -0.008500 -0.341333 -0.005153  ... -0.341333 -0.036589 -0.341333 -2.585066   
2 -0.011830 -0.341333 -0.010238  ... -0.341333 -0.027783 -0.341333 -3.513576   
3 -0.014218 -0.341333 -0.014706  ... -0.341333 -0.019906 -0.341333 -4.341096   
4 -0.015284 -0.341333 -0.017220  ..